In [1]:
import json
import csv
import os
from pprint import pprint as pr
import sqlalchemy as sa
from sqlalchemy import join
import pandas as pd
from app.sample_data import stat_vectors, comp_style_vectors, map_style_vectors, roles

In [2]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [3]:
Player=meta_data.tables['Player']
TeamMember=meta_data.tables['TeamMember']
Hero=meta_data.tables['Hero']
Battleground=meta_data.tables['Map']
Replay=meta_data.tables['Replay']

In [4]:
team_id = 4109
act_list = ['TheCaptain#12345', 'Aly#11717']

j = Replay.join(Player,Player.c.Battletag == Replay.c.Battletag).join(Hero, Replay.c.Hero == Hero.c.HeroName)

query = sa.select(
    Replay
    , Player.c.PlayerId
    , Hero.c.HeroId
    , Hero.c.NewRole
).select_from(j).where(sa.and_(
    Replay.c.GameDate >= '01/01/2022'
    , Player.c.Battletag.in_(act_list)
))

df = pd.read_sql(sql = query,
                 con = engine)
df['Battletag'].drop_duplicates(keep='first', inplace=True)
df.Battletag = 'TheCaptain#12345'
df.shape

(914, 71)

In [37]:
df.dtypes

ReplayId                   int64
SourceReplayId             int64
ReplaySource              object
Battletag                 object
GameDate          datetime64[ns]
                       ...      
YearCode                   int64
MonthCode                  int64
DayCode                    int64
Target                     int32
MapCode                     int8
Length: 78, dtype: object

In [36]:
##Formatting the data
df['GameTypeCode'] = df.GameType.astype('category').cat.codes
df['HeroRoleCode'] = df.NewRole.astype('category').cat.codes
df['MapCode'] = df.GameMap.astype('category').cat.codes
df['YearCode'] = df.GameDate.dt.year
df['MonthCode'] = df.GameDate.dt.month
df['DayCode'] = df.GameDate.dt.dayofweek

##Creating a target to determine outcome
df['Target'] = (df.IsWinner == 1 ).astype('int')

df.head(5)

,ReplayId,SourceReplayId,ReplaySource,Battletag,GameDate,RankedSeason,GameType,GameLength,GameMap,Hero,...,PlayerId,HeroId,NewRole,GameTypeCode,HeroRoleCode,YearCode,MonthCode,DayCode,Target,MapCode
0,2,41465867,HeroesProfile,TheCaptain#12345,2022-01-02 03:04:07,2022 Season 1,Quick Match,1009,Sky Temple,Auriel,...,1,9,Healer,1,1,2022,1,6,0,10
1,4,41466439,HeroesProfile,TheCaptain#12345,2022-01-02 04:34:26,2022 Season 1,Quick Match,1105,Volskaya Foundry,Blaze,...,1,11,Tank,1,5,2022,1,6,0,13
2,208,41645712,HeroesProfile,TheCaptain#12345,2022-01-08 22:41:48,2022 Season 1,Quick Match,804,Blackheart's Bay,Cassia,...,1,13,Ranged Assassin,1,3,2022,1,5,1,2
3,499,41543545,HeroesProfile,TheCaptain#12345,2022-01-05 02:00:00,2022 Season 1,Quick Match,1208,Infernal Shrines,Jaina,...,1,35,Ranged Assassin,1,3,2022,1,2,1,9
4,500,41565424,HeroesProfile,TheCaptain#12345,2022-01-04 00:35:51,2022 Season 1,Quick Match,820,Cursed Hollow,Jaina,...,1,35,Ranged Assassin,1,3,2022,1,1,1,4


In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

In [44]:
rf = RandomForestClassifier(
    n_estimators=50
    , min_samples_split=10
    , random_state=1
)

In [45]:
train = df[df.GameDate < '2022-05-01']
test = df[df.GameDate >= '2022-05-01']

In [97]:
predictors = [
    'HeroId', 'MapCode', 'GameTypeCode', 'HeroRoleCode', 'GameLength'
    , 'HeroLevel', 'Kills', 'Assists', 'Deaths'
    , 'ExperienceContribution'
    , 'FirstToTen', 'Level', 'HeroDamage', 'Healing'
    , 'Takedowns'
    , 'DayCode', 'SelfHealing', 'DamageTaken', 'TimeSpentDead'
]

In [98]:
rf.fit(train[predictors], train["Target"])

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [99]:
preds = rf.predict(test[predictors])

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [121]:
accuracy_score(test['Target'], preds)

0.8282828282828283

In [124]:
combined = pd.DataFrame(dict(actual=test["Target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,168,27
1,41,160


In [125]:
precision_score(test['Target'], preds)

0.8556149732620321

In [126]:
grouped_replays = df.groupby('Hero')
group = grouped_replays.get_group('Jaina')
group

,ReplayId,SourceReplayId,ReplaySource,Battletag,GameDate,RankedSeason,GameType,GameLength,GameMap,Hero,...,PlayerId,HeroId,NewRole,GameTypeCode,HeroRoleCode,YearCode,MonthCode,DayCode,Target,MapCode
3,499,41543545,HeroesProfile,TheCaptain#12345,2022-01-05 02:00:00,2022 Season 1,Quick Match,1208,Infernal Shrines,Jaina,...,1,35,Ranged Assassin,1,3,2022,1,2,1,9
4,500,41565424,HeroesProfile,TheCaptain#12345,2022-01-04 00:35:51,2022 Season 1,Quick Match,820,Cursed Hollow,Jaina,...,1,35,Ranged Assassin,1,3,2022,1,1,1,4
18,1515,41625127,HeroesProfile,TheCaptain#12345,2022-01-09 00:00:13,2022 Season 1,Custom,1018,Infernal Shrines,Jaina,...,1,35,Ranged Assassin,0,3,2022,1,6,0,9
19,1516,41665508,HeroesProfile,TheCaptain#12345,2022-01-11 02:15:34,2022 Season 1,Custom,918,Dragon Shire,Jaina,...,1,35,Ranged Assassin,0,3,2022,1,1,0,5
27,1912,41543052,HeroesProfile,TheCaptain#12345,2022-01-04 03:28:57,2022 Season 1,Storm League,1376,Dragon Shire,Jaina,...,1,35,Ranged Assassin,2,3,2022,1,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,473832,44631726,HeroesProfile,TheCaptain#12345,2022-07-04 19:55:38,,Storm League,1329,Cursed Hollow,Jaina,...,1,35,Ranged Assassin,2,3,2022,7,0,0,4
882,473833,44631941,HeroesProfile,TheCaptain#12345,2022-07-04 20:27:14,,Storm League,1411,Garden of Terror,Jaina,...,1,35,Ranged Assassin,2,3,2022,7,0,1,6
883,473834,44632659,HeroesProfile,TheCaptain#12345,2022-07-04 21:38:23,,Storm League,1536,Tomb of the Spider Queen,Jaina,...,1,35,Ranged Assassin,2,3,2022,7,0,1,11
884,473835,44656365,HeroesProfile,TheCaptain#12345,2022-07-05 23:53:32,,Storm League,1488,Cursed Hollow,Jaina,...,1,35,Ranged Assassin,2,3,2022,7,1,0,4


In [117]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("GameDate")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [118]:
cols = [
      'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
]

new_cols = [f'{c}_Rolling' for c in cols]

In [119]:
rolling_averages(group, cols, new_cols)

,ReplayId,SourceReplayId,ReplaySource,Battletag,GameDate,RankedSeason,GameType,GameLength,GameMap,Hero,...,Vengeance_Rolling,OutnumberedDeaths_Rolling,TeamfightEscapes_Rolling,TeamfightHealing_Rolling,TeamfightDamageTaken_Rolling,TeamfightHeroDamage_Rolling,Multikill_Rolling,PhysicalDamage_Rolling,SpellDamage_Rolling,RegenGlobes_Rolling
28,1913,41543438,HeroesProfile,TheCaptain#12345,2022-01-05 01:35:44,2022 Season 1,Storm League,1144,Towers of Doom,Jaina,...,0.333333,0.666667,0.000000,0.0,17150.333333,36979.000000,3.666667,28277.666667,152424.333333,24.666667
3,499,41543545,HeroesProfile,TheCaptain#12345,2022-01-05 02:00:00,2022 Season 1,Quick Match,1208,Infernal Shrines,Jaina,...,1.000000,0.666667,0.000000,0.0,16304.000000,35169.000000,2.333333,20039.000000,157784.666667,26.666667
29,1914,41556021,HeroesProfile,TheCaptain#12345,2022-01-06 01:22:38,2022 Season 1,Storm League,1058,Tomb of the Spider Queen,Jaina,...,1.000000,1.000000,0.000000,0.0,20349.333333,36474.000000,1.333333,18388.000000,165087.000000,29.666667
30,1915,41556747,HeroesProfile,TheCaptain#12345,2022-01-06 02:45:05,2022 Season 1,Storm League,913,Dragon Shire,Jaina,...,1.000000,2.000000,0.000000,0.0,15781.000000,25868.666667,1.000000,16936.000000,128147.666667,27.000000
31,1919,41591364,HeroesProfile,TheCaptain#12345,2022-01-07 18:05:20,2022 Season 1,Storm League,1783,Tomb of the Spider Queen,Jaina,...,0.333333,1.666667,0.000000,0.0,14877.666667,22303.666667,2.000000,16223.666667,127369.000000,27.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,473845,44556143,HeroesProfile,TheCaptain#12345,2022-06-29 02:07:03,,Custom,912,Towers of Doom,Jaina,...,0.333333,0.666667,0.000000,0.0,13266.333333,21514.666667,2.000000,20656.333333,141567.000000,28.666667
881,473832,44631726,HeroesProfile,TheCaptain#12345,2022-07-04 19:55:38,,Storm League,1329,Cursed Hollow,Jaina,...,0.000000,1.333333,0.000000,0.0,13230.000000,24658.000000,2.333333,13262.000000,119802.000000,20.333333
882,473833,44631941,HeroesProfile,TheCaptain#12345,2022-07-04 20:27:14,,Storm League,1411,Garden of Terror,Jaina,...,0.000000,1.000000,0.000000,0.0,15243.333333,29666.000000,2.333333,16668.666667,167064.333333,23.666667
883,473834,44632659,HeroesProfile,TheCaptain#12345,2022-07-04 21:38:23,,Storm League,1536,Tomb of the Spider Queen,Jaina,...,0.333333,1.333333,0.333333,0.0,16744.000000,32991.666667,2.666667,13302.666667,171340.333333,24.333333


In [143]:
replay_rolling = df.groupby('Hero').apply(lambda x: rolling_averages(x, cols, new_cols)).droplevel('Hero')
replay_rolling.index = range(replay_rolling.shape[0]) # Gives unique values for each row in the index.
replay_rolling

,ReplayId,SourceReplayId,ReplaySource,Battletag,GameDate,RankedSeason,GameType,GameLength,GameMap,Hero,...,Vengeance_Rolling,OutnumberedDeaths_Rolling,TeamfightEscapes_Rolling,TeamfightHealing_Rolling,TeamfightDamageTaken_Rolling,TeamfightHeroDamage_Rolling,Multikill_Rolling,PhysicalDamage_Rolling,SpellDamage_Rolling,RegenGlobes_Rolling
0,443503,43244919,HeroesProfile,TheCaptain#12345,2022-04-10 05:10:39,2022 Season 1,Quick Match,1035,Warhead Junction,Anub'arak,...,0.000000,1.333333,0.0,0.0,39151.000000,18148.666667,1.000000,17988.333333,44648.333333,20.000000
1,443547,43378545,HeroesProfile,TheCaptain#12345,2022-04-18 22:04:12,2022 Season 1,Storm League,1226,Hanamura Temple,Anub'arak,...,0.333333,2.333333,0.0,0.0,48692.000000,18131.000000,1.000000,18172.666667,51770.333333,19.666667
2,443548,43659098,HeroesProfile,TheCaptain#12345,2022-05-06 00:59:11,2022 Season 1,Storm League,964,Dragon Shire,Anub'arak,...,0.333333,2.000000,0.0,0.0,49237.666667,17560.333333,1.666667,16903.666667,52004.000000,20.666667
3,442831,43799922,HeroesProfile,TheCaptain#12345,2022-05-15 03:40:15,2022 Season 1,Quick Match,1418,Tomb of the Spider Queen,Anub'arak,...,0.333333,2.000000,0.0,0.0,40584.666667,12996.000000,1.666667,21576.000000,64421.000000,21.666667
4,443504,43801509,HeroesProfile,TheCaptain#12345,2022-05-15 06:31:43,2022 Season 1,Quick Match,1136,Braxis Holdout,Anub'arak,...,0.000000,1.333333,0.0,0.0,50201.666667,23204.666667,1.333333,22235.000000,76938.666667,25.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,443009,43596838,HeroesProfile,TheCaptain#12345,2022-05-03 02:04:28,2022 Season 1,Storm League,989,Sky Temple,Zarya,...,1.666667,1.333333,0.0,0.0,24820.333333,27937.333333,4.666667,71928.333333,44196.000000,32.000000
783,443025,43612427,HeroesProfile,TheCaptain#12345,2022-05-04 01:05:44,2022 Season 1,Custom,827,Towers of Doom,Zarya,...,1.666667,1.000000,0.0,0.0,29117.000000,30596.000000,3.666667,73210.333333,41875.000000,29.666667
784,443010,43659087,HeroesProfile,TheCaptain#12345,2022-05-04 19:29:10,2022 Season 1,Storm League,956,Hanamura Temple,Zarya,...,0.666667,1.333333,0.0,0.0,29049.000000,23433.000000,2.000000,58438.666667,39014.333333,25.000000
785,443011,43744097,HeroesProfile,TheCaptain#12345,2022-05-07 22:14:32,2022 Season 1,Storm League,1535,Alterac Pass,Zarya,...,0.333333,2.000000,0.0,0.0,32242.666667,23415.333333,1.000000,52695.333333,39009.000000,24.000000


In [144]:
def make_predictions(data, predictors, rand_f):
    train = data[data.GameDate < '2022-05-01']
    test = data[data.GameDate >= '2022-05-01']
    rand_f.fit(train[predictors], train["Target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["Target"], prediction=preds), index=test.index)
    precision = precision_score(test['Target'], preds)
    return combined, precision

In [145]:
predictors = [
    'HeroId', 'MapCode', 'GameTypeCode', 'HeroRoleCode', 'GameLength'
    , 'HeroLevel', 'Kills', 'Assists', 'Deaths'
    , 'ExperienceContribution', 'FirstToTen', 'Level', 'HeroDamage', 'Healing'
    , 'Takedowns', 'DayCode', 'SelfHealing', 'DamageTaken', 'TimeSpentDead'
]

cols = [
      'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
]
new_cols = [f'{c}_Rolling' for c in cols]
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf.fit(train[predictors], train["Target"])
combined, precision = make_predictions(replay_rolling, predictors + new_cols, rf)
precision

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.8432432432432433

In [146]:
combined = combined.merge(replay_rolling[["GameDate", "Battletag", "Hero", "IsWinner"]], left_index=True, right_index=True)
combined

,actual,prediction,GameDate,Battletag,Hero,IsWinner
2,1,1,2022-05-06 00:59:11,TheCaptain#12345,Anub'arak,1
3,1,0,2022-05-15 03:40:15,TheCaptain#12345,Anub'arak,1
4,1,1,2022-05-15 06:31:43,TheCaptain#12345,Anub'arak,1
5,0,0,2022-05-21 21:38:43,TheCaptain#12345,Anub'arak,0
6,0,0,2022-05-30 19:58:26,TheCaptain#12345,Anub'arak,0
...,...,...,...,...,...,...
782,1,1,2022-05-03 02:04:28,TheCaptain#12345,Zarya,1
783,0,0,2022-05-04 01:05:44,TheCaptain#12345,Zarya,0
784,0,0,2022-05-04 19:29:10,TheCaptain#12345,Zarya,0
785,0,1,2022-05-07 22:14:32,TheCaptain#12345,Zarya,0


In [13]:
# df.columns
points = [
    'Battletag', 'Hero', 'NewRole', 'GameMap'
    , 'GameDate',  'GameType', 'GameLength'
    , 'HeroLevel', 'Kills', 'Assists', 'Takedowns', 'Deaths', 'Team'
    , 'IsWinner', 'FirstToTen', 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'MatchAward', 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    , 'PlayerId','HeroId', 'ReplayId', 'SourceReplayId', 'Party', 'RankedSeason'
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]

agg_points = [
    'Battletag', 'Hero', 'NewRole', 'GameLength'
    , 'HeroLevel', 'Kills', 'Assists', 'Takedowns', 'Deaths', 'Team'
    , 'IsWinner', 'FirstToTen', 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'MatchAward', 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    # , 'GameDate',  'GameType'
    # , 'PlayerId','HeroId',
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]


dps_agg_points = [
    'Battletag', 'Hero', 'NewRole', 'IsWinner'
    , 'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    # , 'GameDate',  'GameType'
    # , 'PlayerId','HeroId',
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]

In [14]:
df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg(
    MeanKills=('Kills', 'mean'), KillCount=('Kills', 'count'), MedianKills=('Kills', 'median'),
    Takedowns=('Takedowns', 'count')
)

MeanKills  KillCount  \
Battletag        Hero        NewRole                                 
TheCaptain#12345 Anub'arak   Tank              1.600000          5   
                 Auriel      Healer            0.000000          1   
                 Blaze       Tank              1.000000          1   
                 Brightwing  Healer            2.000000          8   
                 Cassia      Ranged Assassin   4.405797         69   
                 Chromie     Ranged Assassin   3.222222          9   
                 Deathwing   Bruiser           4.500000          2   
                 Deckard     Healer            0.333333          3   
                 Dehaka      Bruiser           0.000000          1   
                 Diablo      Tank              1.666667          3   
                 Fenix       Ranged Assassin   2.500000          2   
                 Gall        Ranged Assassin   1.500000          2   
                 Gazlowe     Bruiser           0.000000          1   
                 Genji       Ranged Assassin   8.000000          2   
                 Greymane    Ranged Assassin   5.000000          1   
                 Gul'dan     Ranged Assassin   3.055556         18   
                 Jaina       Ranged Assassin   4.320261        153   
                 Johanna     Tank              2.875000          8   
                 Kael'thas   Ranged Assassin   4.541667         24   
                 Kerrigan    Melee Assassin    4.000000          1   
                 Kharazim    Healer            3.333333          3   
                 Li-Ming     Ranged Assassin   4.000000         35   
                 Lt. Morales Healer            1.000000          3   
                 Mal'Ganis   Tank              4.000000          1   
                 Malfurion   Healer            1.666667          3   
                 Mephisto    Ranged Assassin   1.900000         10   
                 Nazeebo     Ranged Assassin   3.000000         14   
                 Orphea      Ranged Assassin   4.543478         46   
                 Probius     Ranged Assassin   2.000000         30   
                 Qhira       Melee Assassin    3.900000         10   
                 Raynor      Ranged Assassin   4.000000          1   
                 Rehgar      Healer            0.500000          2   
                 Sgt. Hammer Ranged Assassin   2.750000          4   
                 Stitches    Tank              2.333333          3   
                 Stukov      Healer            3.257143         35   
                 Sylvanas    Ranged Assassin   2.000000          2   
                 The Butcher Melee Assassin    3.000000          2   
                 Thrall      Bruiser           4.000000          5   
                 Tyrande     Healer            0.500000          2   
                 Uther       Healer            2.000000          2   
                 Valeera     Melee Assassin    5.800000          5   
                 Valla       Ranged Assassin   4.594595         37   
                 Varian      Bruiser           4.310811         74   
                 Whitemane   Healer            1.000000          2   
                 Zagara      Ranged Assassin   5.000000          1   
                 Zarya       Support           3.750000          8   
                 Zul'jin     Ranged Assassin   4.000000          2   

                                              MedianKills  Takedowns  
Battletag        Hero        NewRole                                  
TheCaptain#12345 Anub'arak   Tank                     1.0          5  
                 Auriel      Healer                   0.0          1  
                 Blaze       Tank                     1.0          1  
                 Brightwing  Healer                   1.0          8  
                 Cassia      Ranged Assassin          4.0         69  
                 Chromie     Ranged Assassin          1.0          9  
                 Deathwing   Bruiser           

In [224]:
df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg([ "max", "sum", "count", "mean"], axis=0)

IsWinner                     Kills  \
                                            max sum count      mean   max   
Battletag    Hero      NewRole                                              
Aly#11717    Anub'arak Tank                   1  10    12  0.833333     3   
             Cassia    Ranged Assassin        1  26    44  0.590909    10   
             Dehaka    Bruiser                0   0     4  0.000000     1   
             Diablo    Tank                   0   0     2  0.000000     3   
             Fenix     Ranged Assassin        0   0     2  0.000000     0   
...                                         ...  ..   ...       ...   ...   
YipYip#11141 Imperius  Bruiser                0   0     2  0.000000     0   
             Lunara    Ranged Assassin        1   8    16  0.500000     9   
             Raynor    Ranged Assassin        1  12    16  0.750000     4   
             Sylvanas  Ranged Assassin        1   4    14  0.285714     7   
             Valla     Ranged Assassin        1   6    14  0.428571    15   

                                                            Assists       ...  \
                                        sum count      mean     max  sum  ...   
Battletag    Hero      NewRole                                            ...   
Aly#11717    Anub'arak Tank              20    12  1.666667      13  100  ...   
             Cassia    Ranged Assassin  156    44  3.545455      23  408  ...   
             Dehaka    Bruiser            2     4  0.500000      11   30  ...   
             Diablo    Tank               6     2  3.000000      11   22  ...   
             Fenix     Ranged Assassin    0     2  0.000000       3    6  ...   
...                                     ...   ...       ...     ...  ...  ...   
YipYip#11141 Imperius  Bruiser            0     2  0.000000      11   22  ...   
             Lunara    Ranged Assassin   84    16  5.250000      24  162  ...   
             Raynor    Ranged Assassin   34    16  2.125000      17  124  ...   
             Sylvanas  Ranged Assassin   68    14  4.857143      18  160  ...   
             Valla     Ranged Assassin   80    14  5.714286      12  104  ...   

                                       PhysicalDamage                 \
                                                count           mean   
Battletag    Hero      NewRole                                         
Aly#11717    Anub'arak Tank                        12   19936.333333   
             Cassia    Ranged Assassin             44   32339.227273   
             Dehaka    Bruiser                      4   22302.000000   
             Diablo    Tank                         2   27723.000000   
             Fenix     Ranged Assassin              2   56330.000000   
...                                               ...            ...   
YipYip#11141 Imperius  Bruiser                      2   34818.000000   
             Lunara    Ranged Assassin             16   50279.500000   
             Raynor    Ranged Assassin             16  105575.750000   
             Sylvanas  Ranged Assassin             14   76491.428571   
             Valla     Ranged Assassin             14   73247.142857   

                                       SpellDamage                 \
                                               max      sum count   
Battletag    Hero      NewRole                                      
Aly#11717    Anub'arak Tank                  90282   654482    12   
             Cassia    Ranged Assassin      195268  4506456    44   
             Dehaka    Bruiser               66791   264456     4   
             Diablo    Tank                  81278   162556     2   
             Fenix     Ranged Assassin       34956    69912     2   
...                                            ...      ...   ...   
YipYip#11141 Imperius  Bruiser               47113    94226     2   
             Lunara    Ranged Assassin      136293  1606160    16   
             Raynor    Ranged Assassin       45022   3925

In [225]:
dfn = df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg(
    HeroCount=('Kills', 'count')
    , Wins=('IsWinner', 'sum')
).sort_values('HeroCount', ascending=False)

dfn['WinRate'] = round(dfn.Wins / dfn.HeroCount, 2)

dfn.groupby(['Battletag']).head(5).sort_index()
# df.sort_values('Count', ascending=False).groupby('Date').head(2).sort_index()

HeroCount  Wins  WinRate
Battletag         Hero       NewRole                                  
Aly#11717         Cassia     Ranged Assassin         44    26     0.59
                  Jaina      Ranged Assassin         80    52     0.65
                  Orphea     Ranged Assassin         86    48     0.56
                  Valla      Ranged Assassin         22     4     0.18
                  Varian     Bruiser                 50    26     0.52
Anaressa#1611     Johanna    Tank                    52    24     0.46
                  Lunara     Ranged Assassin        116    84     0.72
                  Raynor     Ranged Assassin         52    30     0.58
                  Sylvanas   Ranged Assassin        124    64     0.52
                  Valla      Ranged Assassin         80    42     0.52
BornToShine#11556 Anduin     Healer                 218   116     0.53
                  Brightwing Healer                 180    84     0.47
                  Deckard    Healer                  40    24     0.60
                  Rehgar     Healer                 154    86     0.56
                  Stukov     Healer                 224   122     0.54
DirePants#125315  Anub'arak  Tank                    86    48     0.56
                  Dehaka     Bruiser                 40    22     0.55
                  E.T.C.     Tank                    56    26     0.46
                  Johanna    Tank                    64    44     0.69
                  Leoric     Bruiser                 42    32     0.76
Direpants#125315  Diablo     Tank                     2     2     1.00
                  Malthael   Bruiser                  2     2     1.00
Knivers#1675      Anub'arak  Tank                    42    24     0.57
                  Blaze      Tank                    30    18     0.60
                  E.T.C.     Tank                    30    18     0.60
                  Garrosh    Tank                    48    28     0.58
                  Muradin    Tank                    34    18     0.53
MowBWan#1362      Anub'arak  Tank                    69    35     0.51
                  Diablo     Tank                    72    29     0.40
                  Johanna    Tank                   169    87     0.51
                  Muradin    Tank                   156    89     0.57
                  Sonya      Bruiser                 31    20     0.65
PapaShango#11954  Artanis    Bruiser                  2     0     0.00
SubZero#1577      Greymane   Ranged Assassin        104    64     0.62
                  Sonya      Bruiser                 32    22     0.69
                  Tassadar   Ranged Assassin         58    22     0.38
                  Tychus     Ranged Assassin         90    54     0.60
                  Zul'jin    Ranged Assassin         84    46     0.55
TheCaptain#12345  Cassia     Ranged Assassin        112    72     0.64
                  Jaina      Ranged Assassin        272   156     0.57
                  Orphea     Ranged Assassin         90    46     0.51
                  Valla      Ranged Assassin         72    32     0.44
                  Varian     Bruiser                 98    44     0.45
YipYip#11141      Cassia     Ranged Assassin         10     6     0.60
                  Lunara     Ranged Assassin         16     8     0.50
                  Raynor     Ranged Assassin         16    12     0.75
                  Sylvanas   Ranged Assassin         14     4     0.29
                  Valla      Ranged Assassin         14     6     0.43